# ANOVA
This will have to wait on processing the Nucleus.csv files.

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
print(datetime.now())


2022-07-21 14:45:25.542625


In [ ]:
BASE_PATH_IN='/home/jrm/Adjeroh/Naved/CP_80K/'
INPUT_DIRS=[
'Output5/',
'Output4/',
'Output3/',
'Output2/',
'Output1/',
'Output0/'
]
FILENAMES=[
'Process100_Image.csv',
'Process100_Cells.csv',
'Process100_ExpandCells.csv',
'Process100_Experiment.csv',
'Process100_MergeRBC.csv',
'Process100_Nucleus.csv',
'Process100_RBC.csv',
'Process100_ShrinkRBC.csv',
'Process100_Tissue.csv']
TUMOR_COL='FileName_Tumor'   # use this column to disambiguate patients
PATIENT_COL='Patient'        # add this column to emphasize patient ID